# AdaptiveGraph Interactive Demo

This notebook demonstrates how to use `adaptivegraph` with `langgraph` to create a self-optimizing workflow.

We will build a simple routing agent that learns to send "VIP" users to a premium model and "Guest" users to a fast model.

In [ ]:
# Install local package in editable mode with dev dependencies
# This installs 'adaptivegraph' along with 'matplotlib' and other dev tools defined in pyproject.toml
# NOTE: We use '..' because this notebook is inside the 'notebooks/' directory
%pip install -e "..[all]"

In [1]:
import random
import matplotlib.pyplot as plt
from typing import Literal, TypedDict
from langgraph.graph import StateGraph, END
from adaptivegraph import LearnableEdge

# Define our state
class AgentState(TypedDict):
    user_type: str
    query: str
    path_taken: str
    outcome: float

In [2]:
# Define Nodes

def start_node(state: AgentState):
    print(f"Processing request for {state['user_type']}...")
    return state

def premium_model(state: AgentState):
    return {"path_taken": "premium", "outcome": 1.0 if state["user_type"] == "vip" else 0.0}

def fast_model(state: AgentState):
    return {"path_taken": "fast", "outcome": 1.0 if state["user_type"] == "guest" else 0.0}

In [3]:
# Define the Learnable Edge

router = LearnableEdge(
    options=["premium", "fast"],
    policy="linucb",
    feature_dim=16,
    exploration_alpha=0.5,
    value_key="user_type" # Extract this key from state automatically
)
# No wrapper needed anymore!

In [4]:
# Build the Graph
workflow = StateGraph(AgentState)

workflow.add_node("start", start_node)
workflow.add_node("premium", premium_model)
workflow.add_node("fast", fast_model)

workflow.set_entry_point("start")

# Add conditional edge directly!
workflow.add_conditional_edges(
    "start",
    router,
    {
        "premium": "premium",
        "fast": "fast"
    }
)

workflow.add_edge("premium", END)
workflow.add_edge("fast", END)

app = workflow.compile()

In [5]:
# Simulation Loop
history = []
accuracies = []

for i in range(100):
    # Generate synthetic data
    u_type = "vip" if random.random() < 0.5 else "guest"
    initial_state = {"user_type": u_type, "query": "hello", "path_taken": "", "outcome": 0.0}
    
    # Run Graph
    result = app.invoke(initial_state)
    
    # Feedback Loop
    # In this toy example, the nodes themselves calculated the 'outcome' (reward)
    reward = result["outcome"]
    
    # CRITICAL: Teach the router!
    router.record_feedback(result, reward=reward)
    
    history.append(reward)
    avg_acc = sum(history[-20:]) / len(history[-20:])
    accuracies.append(avg_acc)
    
    if i % 10 == 0:
        print(f"Step {i}: Type={u_type}, Path={result['path_taken']}, Reward={reward}")

print(f"Final Accuracy (last 20): {accuracies[-1]:.2f}")

In [6]:
# Plot Learning Curve
plt.plot(accuracies)
plt.title("Routing Accuracy over Time")
plt.xlabel("Iterations")
plt.ylabel("Moving Average Accuracy")
plt.ylim(0, 1.1)
plt.show()

## Advanced: Semantic Routing (Batteries Included)

If you have `sentence-transformers` and `faiss-cpu` installed (included in `[all]`), you can use the factory method to create a semantic router out of the box.

In [5]:
try:
    # Create a semantic router
    # This one will embed the input text and find the best action based on similarity/bandits
    semantic_edge = LearnableEdge.create(
        options=["technical_support", "billing", "general_chat"],
        embedding="sentence-transformers",
        memory="faiss",
        feature_dim=384, # Standard for all-MiniLM-L6-v2
        exploration_alpha=0.1
    )
    print("Semantic Edge created successfully!")
except ImportError:
    print("Please install 'sentence-transformers' and 'faiss-cpu' to run this section.")

Semantic Edge created successfully!


In [6]:
# Quick Test
queries = [
    ("My screen is black and won't turn on", "technical_support"),
    ("I need to pay my invoice", "billing"),
    ("How are you doing today?", "general_chat")
]

print("Training on a few examples...")
for i in range(20):
    q, target = random.choice(queries)
    
    choice = semantic_edge(q)
    reward = 1.0 if choice == target else 0.0
    
    semantic_edge.record_feedback({}, reward=reward)
    
    if i % 5 == 0:
        print(f"'{q}' -> {choice} (Reward: {reward})")

print("\nFinal Check:")
print(f" Invoice -> {semantic_edge('Where do I send the check?')}")
print(f" Broken -> {semantic_edge('System crash error 505')}")

Training on a few examples...
'I need to pay my invoice' -> general_chat (Reward: 0.0)
'How are you doing today?' -> technical_support (Reward: 0.0)
'My screen is black and won't turn on' -> technical_support (Reward: 1.0)
'My screen is black and won't turn on' -> technical_support (Reward: 1.0)

Final Check:
 Invoice -> billing
 Broken -> technical_support
